# Deep Learning for Obstacle Avoidance

This notebook explains how deep learning is used to shape a straight line (street) to avoid obstacles (holes, geofences, etc.).

## Vehicle Footprint and Collision Avoidance

The algorithm considers that the path is followed by a vehicle with a specific **footprint** (a rectangular shape representing the machine). The goal is to ensure that this footprint does not collide with obstacles (holes, geofence boundaries), which also have a safety **buffer** range.

The logic checks for collisions by generating the vehicle's footprint at each point along the path and verifying if it intersects with any buffered obstacle.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import math
import random
from sklearn.decomposition import PCA
import shapely
import geopandas as gpd
from shapely.geometry import LineString, Point, Polygon
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
import traceback
warnings.filterwarnings('ignore')

print(f'Shapely version: {shapely.__version__}')
print(f'Torch version: {torch.__version__}')

Shapely version: 2.0.0
Torch version: 2.4.1+cu121


## Street Fitting Logic

The following code is the core logic for fitting streets. It includes the `gen_footprint_obstacle` function which defines the vehicle's shape and `del_colliding` which helps in avoiding collisions.

In [2]:

# Global Parameters (Controlled by Sliders)
HOLE_BUFFER = 7.0
FOOTPRINT_FRONT = 3.5
FOOTPRINT_BACK = -3.5
FOOTPRINT_SIDE = 1.8
FOOTPRINT_BUFFER = 1.0

# Optimization Tuning Parameters
REPULSION_WEIGHT = 500.0
FIDELITY_WEIGHT = 0.0001
SAFETY_MARGIN = 0.5
SOFT_LIMIT_RANGE = 5.0
ITERATIONS = 500


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import math
import random
from sklearn.decomposition import PCA
from IPython.display import display
import pandas as pd
import geopandas as gpd
import shapely


def translate(xarr, yarr, tx, ty):
    xret = []
    yret = []
    for i in range(len(xarr)):
        x = xarr[i] + tx
        y = yarr[i] + ty
        xret.append(x)
        yret.append(y)
    return xret, yret


def rotate(xarr, yarr, angle):
    xret = []
    yret = []
    for i in range(len(xarr)):
        x = math.cos(angle) * xarr[i] - math.sin(angle) * yarr[i]
        y = math.sin(angle) * xarr[i] + math.cos(angle) * yarr[i]
        xret.append(x)
        yret.append(y)
    return xret, yret


def get_pca_transform(xarr,yarr):
    n = len(xarr)
    X = []
    sx = 0
    sy = 0
    for i in range(n):
        X.append((xarr[i],yarr[i]))
        sx += xarr[i]
        sy += yarr[i]
    pca = PCA(n_components=2)
    X_fit = pca.fit(X)
    angle = math.atan2(pca.components_[0,1], pca.components_[0,0])
    return sx/n, sy/n, angle


def select_near_holes(xarr, yarr, holesx, holesy, d_thr):
    curve_xy = [(xarr[i],yarr[i]) for i in range(len(xarr))]
    holes_xy = [(holesx[i],holesy[i]) for i in range(len(holesx))]

    curve = shapely.LineString(curve_xy)
    holes = shapely.MultiPoint(holes_xy)

    street : shapely.Polygon = shapely.buffer(curve, d_thr)
    holes : shapely.MultiPoint = street.intersection(holes)

    xret = []
    yret = []
    xy : shapely.Point

    if not holes.is_empty:
        if type(holes) == shapely.MultiPoint:
            for xy in holes.geoms:
                xret.append(xy.x)
                yret.append(xy.y)
        else:
            xret.append(holes.x)
            yret.append(holes.y)

    return xret, yret


def select_near_geofence_points(xarr, yarr, geofx, geofy, d_thr):
    return select_near_holes(xarr, yarr, geofx, geofy, d_thr)


def sample_polygon(polygon : shapely.Polygon, dist = 1.0):
    boundary : shapely.LineString = polygon.boundary
    length = boundary.length
    points = shapely.line_interpolate_point(boundary, [x for x in np.arange(0, length, dist)])
    xret = []
    yret = []
    xy : shapely.Point
    for xy in points:
        xret.append(xy.x)
        yret.append(xy.y)
    return xret, yret


def my_loss(x, y, yo, xh, yh, xlow, ylow, xhigh, yhigh, safety_radius=2.1):
    loss = 0
    
    # New curve must be near old curve (small loss term)
    # Uses global FIDELITY_WEIGHT
    loss += FIDELITY_WEIGHT * torch.mean((y[1:-1]-yo[1:-1])**2) 
        
    # New curve must have small curvature
    curvature2 = (y[0:-2] + y[2:] - 2*y[1:-1] )**2 / (x[2:] - x[:-2])**2
    loss += 60*torch.mean(curvature2)

    # Penalize strong curvatures larger than that valid for Zeus
    loss += 200*torch.mean((curvature2-0.003)*torch.nn.ReLU()(curvature2-0.003))
    
    # New curve must keep the previous start/end
    loss += 10 * (y[0]-yo[0])**2
    loss += 10 * (y[1]-yo[1])**2
    loss += 10 * (y[-2]-yo[-2])**2
    loss += 10 * (y[-1]-yo[-1])**2
    
    # New curve must be far from cuttings (Holes)
    xh = torch.reshape(xh, (xh.shape[0], 1))
    yh = torch.reshape(yh, (yh.shape[0], 1))
    dx = x - xh
    dy = y - yh
    d2 = dx**2 + dy**2
    
    # Dynamic thresholds based on safety_radius
    hard_limit_sq = safety_radius**2
    
    # Uses global SOFT_LIMIT_RANGE
    soft_limit_sq = (safety_radius + SOFT_LIMIT_RANGE)**2
    
    # Soft repulsion (pushes away gently from distance)
    loss += (0.5 / d2 * torch.sigmoid(5*(soft_limit_sq - d2))).sum()
    
    # AGGRESSIVE Repulsion using ReLU (Hard Wall)
    # Uses global REPULSION_WEIGHT
    violation = torch.nn.ReLU()(hard_limit_sq - d2)
    loss += REPULSION_WEIGHT * torch.mean(violation)

    # Compute primer coordinates (xp,yp)
    L = torch.sqrt( (x[2:] - x[:-2])**2 + (y[2:] - y[:-2])**2)
    C = (x[2:] - x[:-2]) / L   # Cosine of angle
    S = (y[2:] - y[:-2]) / L   # Sine of angle

    xp = x[1:-1] + 3 * C
    yp = y[1:-1] + 3 * S

    # New curve must be far from low obstacles (eval at base_link)
    xlow = torch.reshape(xlow, (xlow.shape[0], 1))
    ylow = torch.reshape(ylow, (ylow.shape[0], 1))
    dx = x - xlow
    dy = y - ylow
    d2 = dx**2 + dy**2
    loss += (20.0 / (d2+0.5) * torch.sigmoid(5*(2.1*2.1 - d2)) ).sum()

    # New curve must be far from high obstacles (eval at base_link)
    xhigh = torch.reshape(xhigh, (xhigh.shape[0], 1))
    yhigh = torch.reshape(yhigh, (yhigh.shape[0], 1))
    dx = x - xhigh
    dy = y - yhigh
    d2 = dx**2 + dy**2
    loss += (20.0 / (d2+0.5) * torch.sigmoid(5*(2.1*2.1 - d2)) ).sum()

    # New curve must be far from high obstacles (eval at primer)
    dx = xp - xhigh[1:-1,:]
    dy = yp - yhigh[1:-1,:]
    d2 = dx**2 + dy**2
    loss += (2.0 / (d2+0.5) * torch.sigmoid(5*(2.1*2.1 - d2)) ).sum()

    return loss


class CurveModel(torch.nn.Module):
  def __init__(self, device, x, y0):
    super(CurveModel, self).__init__()
    self.yb = torch.clone(y0).to(device=device)
    self.y = torch.nn.Parameter(self.yb)
 
  def forward(self, x):
    return self.y



def fit_street(orig_x, orig_y, holes_x, holes_y, low_x, low_y, high_x, high_y, safety_radius=2.1):
    # Ensure inputs are lists for compatibility with helper functions
    if hasattr(orig_x, 'tolist'): orig_x = orig_x.tolist()
    if hasattr(orig_y, 'tolist'): orig_y = orig_y.tolist()
    
    tx, ty, angle = get_pca_transform(orig_x,orig_y)

    orig_x, orig_y = translate(orig_x, orig_y, -tx, -ty)
    orig_x, orig_y = rotate(orig_x, orig_y, -angle)

    holes_x, holes_y = translate(holes_x, holes_y, -tx, -ty)
    holes_x, holes_y = rotate(holes_x, holes_y, -angle)    

    low_x, low_y = translate(low_x, low_y, -tx, -ty)
    low_x, low_y = rotate(low_x, low_y, -angle)    

    high_x, high_y = translate(high_x, high_y, -tx, -ty)
    high_x, high_y = rotate(high_x, high_y, -angle)    

    device = torch.device('cpu:0')

    xo = torch.Tensor(orig_x).to(device)
    yo = torch.Tensor(orig_y).to(device)

    xh = torch.Tensor(holes_x).to(device)
    yh = torch.Tensor(holes_y).to(device)

    xlow = torch.Tensor(low_x).to(device)
    ylow = torch.Tensor(low_y).to(device)

    xhigh = torch.Tensor(high_x).to(device)
    yhigh = torch.Tensor(high_y).to(device)

    model = CurveModel(device, xo, yo)

    #optim = torch.optim.SGD(model.parameters(), lr=0.001)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)

    loss_history = [] 
    path_history = [] # Store intermediate paths
    
    # Use global ITERATIONS
    n_iters = int(ITERATIONS) 
    for i in range(0, n_iters):
        if i == 100:
            for g in optim.param_groups:
                g['lr'] = 0.01

        if i == 200:
            for g in optim.param_groups:
                g['lr'] = 0.03

        predictions = model.forward(xo)
        # Pass safety_radius to my_loss
        loss = my_loss(xo, predictions, yo, xh, yh, xlow, ylow, xhigh, yhigh, safety_radius)
        loss.backward()
        optim.step()
        optim.zero_grad()
        
        loss_history.append(loss.item()) 

        # Capture path every 50 epochs
        if i % 50 == 0:
            curr_y = predictions.detach().cpu().numpy()
            # Transform back to global coordinates for storage
            # Ensure orig_x is list for rotate
            cx, cy = rotate(orig_x, curr_y, angle)
            cx, cy = translate(cx, cy, tx, ty)
            path_history.append((cx, cy))

        if i % 50 == 0:
            # print(f"epoch {i} / {n_iters}   loss {loss.item()}")
            if torch.isnan(predictions).any():
                print("NAN IN OPTIMIZATION")
    
    curve_x = orig_x
    curve_y = predictions.detach().cpu().numpy()

    curve_x, curve_y = rotate(curve_x, curve_y, angle)
    curve_x, curve_y = translate(curve_x, curve_y, tx, ty)

    return curve_x, curve_y, loss_history, path_history # Return path_history



def gen_footprint_obstacle(x,y,angle, buffer_val=None):
    back = FOOTPRINT_BACK
    front = FOOTPRINT_FRONT
    left = FOOTPRINT_SIDE
    right = -FOOTPRINT_SIDE
    c = math.cos(angle)
    s = math.sin(angle)

    footprint = shapely.Polygon([
        [x+c*back -s*left,  y+s*back +c*left],
        [x+c*front-s*left,  y+s*front+c*left],
        [x+c*front-s*right, y+s*front+c*right],
        [x+c*back -s*right, y+s*back +c*right],
        [x+c*back -s*left,  y+s*back +c*left]
    ])
    
    # Use provided buffer_val if given, otherwise use global FOOTPRINT_BUFFER
    buf = buffer_val if buffer_val is not None else FOOTPRINT_BUFFER
    
    if buf > 0:
        footprint = footprint.buffer(buf)
        
    return footprint


def gen_footprint_high_obstacle(x,y,angle):
    back = -3.5
    front = 6.5
    left = 1.8 
    right = -1.8
    c = math.cos(angle)
    s = math.sin(angle)

    footprint = shapely.Polygon([
        [x+c*back -s*left,  y+s*back +c*left],
        [x+c*front-s*left,  y+s*front+c*left],
        [x+c*front-s*right, y+s*front+c*right],
        [x+c*back -s*right, y+s*back +c*right],
        [x+c*back -s*left,  y+s*back +c*left]
    ])    
    return footprint




def del_colliding(orig_x, orig_y, holes_x, holes_y, low_x, low_y, high_x, high_y):
    tx, ty, angle = get_pca_transform(orig_x,orig_y)

    orig_x, orig_y = translate(orig_x, orig_y, -tx, -ty)
    orig_x, orig_y = rotate(orig_x, orig_y, -angle)

    holes_x, holes_y = translate(holes_x, holes_y, -tx, -ty)
    holes_x, holes_y = rotate(holes_x, holes_y, -angle)    

    low_x, low_y = translate(low_x, low_y, -tx, -ty)
    low_x, low_y = rotate(low_x, low_y, -angle)    

    high_x, high_y = translate(high_x, high_y, -tx, -ty)
    high_x, high_y = rotate(high_x, high_y, -angle)    

    # Buffer holes by HOLE_BUFFER + FOOTPRINT_BUFFER
    # Note: We use the global variables
    total_buffer = HOLE_BUFFER + FOOTPRINT_BUFFER
    
    # Fix bug in original code: holes_x[i], holes_y[i]
    holes = shapely.MultiPoint([[holes_x[i],holes_y[i]] for i in range(len(holes_x))]).buffer(total_buffer)
    
    low = shapely.MultiPoint([[low_x[i],low_y[i]] for i in range(len(low_x))])
    high = shapely.MultiPoint([[high_x[i],high_y[i]] for i in range(len(high_x))])

    curve_x = []
    curve_y = []

    for i in range(len(orig_x)):
        if i > 0:
            ori = math.atan2(orig_y[i]-orig_y[i-1], orig_x[i]-orig_x[i-1])
        else:
            ori = math.atan2(orig_y[i+1]-orig_y[i], orig_x[i+1]-orig_x[i])
            
        # Check intersection with RAW footprint (since holes are already buffered by total_buffer)
        footprint = gen_footprint_obstacle(orig_x[i], orig_y[i], ori, buffer_val=0.0)
        
        footprint_high = gen_footprint_high_obstacle(orig_x[i], orig_y[i], ori)
        
        if not holes.intersects(footprint) and not low.intersects(footprint) and not high.intersects(footprint_high):
            curve_x.append(orig_x[i])
            curve_y.append(orig_y[i])

    curve_x, curve_y = rotate(curve_x, curve_y, angle)
    curve_x, curve_y = translate(curve_x, curve_y, tx, ty)

    return curve_x, curve_y



def segmentize_line(line_geom, max_segment_length=1.0):
    # Custom implementation compatible with older Shapely
    length = line_geom.length
    num_points = int(math.ceil(length / max_segment_length)) + 1
    distances = np.linspace(0, length, num_points)
    points = [line_geom.interpolate(d) for d in distances]
    return shapely.LineString(points)

def fit_all_streets(streets : gpd.GeoDataFrame, holes : gpd.GeoDataFrame, geofence : gpd.GeoDataFrame, obstacles : gpd.GeoDataFrame, high_obstacles : gpd.GeoDataFrame, fit_twice : bool, progress_callback=None):
    all_holes_x = holes['x'].to_list()
    all_holes_y = holes['y'].to_list()

    all_geof_x, all_geof_y = sample_polygon(geofence.geometry.iloc[0])
    low_x = []
    low_y = []

    if obstacles is not None:
        for row in obstacles.geometry:
            cx, cy = sample_polygon(row)
            low_x += cx
            low_y += cy

    if high_obstacles is not None:
        for row in high_obstacles.geometry:
            cx, cy = sample_polygon(row)
            all_geof_x += cx
            all_geof_y += cy
            
    # Calculate safety radius for optimization
    # Uses global SAFETY_MARGIN
    safety_radius = HOLE_BUFFER + FOOTPRINT_BUFFER + FOOTPRINT_SIDE + SAFETY_MARGIN

    total_streets = len(streets)
    last_losses = [] 
    last_path_history = []
    
    for index, row in streets.iterrows():
        # Use custom segmentize_line instead of shapely.segmentize
        street = segmentize_line(row['geometry'], max_segment_length=1.0)
        orig_x = []
        orig_y = []
        for xy in list(street.coords):
            orig_x.append(xy[0])
            orig_y.append(xy[1])
        
        # Select holes near the street (using a generous buffer to be safe)
        holes_x, holes_y = select_near_holes(orig_x, orig_y, all_holes_x, all_holes_y, safety_radius + 5.0)
        geof_x, geof_y = select_near_geofence_points(orig_x, orig_y, all_geof_x, all_geof_y, 11.0)

        curve_x, curve_y, losses, path_hist = fit_street(orig_x, orig_y, holes_x, holes_y, low_x, low_y, geof_x, geof_y, safety_radius=safety_radius)
        last_losses = losses
        last_path_history = path_hist

        if fit_twice:
            curve_x_filtered, curve_y_filtered = del_colliding(curve_x, curve_y, holes_x, holes_y, low_x, low_y, geof_x, geof_y)
            
            # CHECK: If del_colliding removed too many points, skip the second fit
            if len(curve_x_filtered) < 5:
                print(f"Warning: del_colliding removed too many points ({len(curve_x_filtered)} left). Skipping second fit.")
                # Keep the result from the first fit
            else:
                curve_x, curve_y = curve_x_filtered, curve_y_filtered
                curve_x, curve_y, losses, path_hist = fit_street(curve_x, curve_y, holes_x, holes_y, low_x, low_y, geof_x, geof_y, safety_radius=safety_radius)
                last_losses = losses
                last_path_history = path_hist

        xy = []
        for i in range(len(curve_x)):
            xy.append( (curve_x[i], curve_y[i]) )
        
        streets.loc[index, 'geometry'] = shapely.LineString(xy)
        streets.loc[index, 'geometry'] = shapely.simplify(streets.loc[index, 'geometry'], 0.1)

    return streets, last_losses, last_path_history


## Demonstration

Let's create a simple scenario with a straight street and some obstacles.

In [3]:

# Create a straight street
street_coords = [(0, 0), (100, 0)]
street_geom = LineString(street_coords)
streets = gpd.GeoDataFrame({'geometry': [street_geom]})

# Create a geofence (large box)
geofence_geom = Polygon([(-10, -20), (110, -20), (110, 20), (-10, 20)])
geofence = gpd.GeoDataFrame({'geometry': [geofence_geom]})

# Function to plot footprints along the path
def plot_footprints(street_geom, ax, step=5):
    coords = list(street_geom.coords)
    for i in range(0, len(coords)-1, step):
        p1 = coords[i]
        p2 = coords[i+1]
        angle = math.atan2(p2[1] - p1[1], p2[0] - p1[0])
        
        # 1. Plot Real Robot (Buffer = 0)
        real_footprint = gen_footprint_obstacle(p1[0], p1[1], angle, buffer_val=0.0)
        x, y = real_footprint.exterior.xy
        ax.plot(x, y, color='purple', alpha=0.1, linewidth=1.0) # Faint solid line
        
        # 2. Plot Safety Buffer (if exists)
        if FOOTPRINT_BUFFER > 0:
            buffered_footprint = gen_footprint_obstacle(p1[0], p1[1], angle) # Uses global buffer
            x_buf, y_buf = buffered_footprint.exterior.xy
            ax.plot(x_buf, y_buf, color='purple', alpha=0.05, linestyle='--') # Very faint dashed line


## Interactive Simulation

Use the sliders below to adjust parameters:

*   **Hole Positions**: Move the obstacles.
*   **Hole Buffer**: Safety distance around holes.
*   **Footprint Dimensions**: Adjust the size of the vehicle (Front, Back, Side).
*   **Footprint Buffer**: Extra safety margin around the vehicle.
*   **Path Position**: Move the robot along the generated path to inspect clearances.
*   **Optimization Tuning**: Adjust weights and margins for the deep learning algorithm.

**Click 'Optimize Route'** to run the deep learning algorithm with the current settings.

In [4]:

# Sliders for Holes
h1_slider = widgets.FloatSlider(min=-15, max=15, step=0.5, value=5, description='Hole 1 Y')
h2_slider = widgets.FloatSlider(min=-15, max=15, step=0.5, value=-5, description='Hole 2 Y')
h3_slider = widgets.FloatSlider(min=-15, max=15, step=0.5, value=5, description='Hole 3 Y')

# Sliders for Parameters
hole_buffer_slider = widgets.FloatSlider(min=1.0, max=15.0, step=0.5, value=7.0, description='Hole Buffer')
fp_front_slider = widgets.FloatSlider(min=1.0, max=10.0, step=0.5, value=3.5, description='FP Front')
fp_back_slider = widgets.FloatSlider(min=-10.0, max=-1.0, step=0.5, value=-3.5, description='FP Back')
fp_side_slider = widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=1.8, description='FP Side')
fp_buffer_slider = widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=1.0, description='FP Buffer')

# Slider for Path Traversal
path_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.0, description='Path Pos')

# Sliders for Optimization Tuning
repulsion_slider = widgets.FloatSlider(min=10.0, max=1000.0, step=10.0, value=500.0, description='Repulsion W')
fidelity_slider = widgets.FloatLogSlider(value=0.0001, base=10, min=-5, max=-1, step=0.1, description='Fidelity W')
safety_margin_slider = widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=0.5, description='Safety Margin')
soft_limit_slider = widgets.FloatSlider(min=1.0, max=10.0, step=0.5, value=5.0, description='Soft Limit')
iterations_slider = widgets.IntSlider(min=100, max=1000, step=50, value=500, description='Iterations')

run_btn = widgets.Button(description="Optimize Route", button_style='success', icon='check')
out = widgets.Output()
debug_out = widgets.Output() # Separate debug output

# Global variable to store the last fitted street and losses
current_fitted_streets = None
current_losses = []
current_path_history = []

def get_holes(h1, h2, h3):
    holes_coords = [(30, h1), (50, h2), (70, h3)]
    holes_geoms = [Point(x, y) for x, y in holes_coords]
    return gpd.GeoDataFrame({'geometry': holes_geoms, 'x': [p[0] for p in holes_coords], 'y': [p[1] for p in holes_coords]})

def plot_current_state(change=None):
    global current_fitted_streets, current_losses, current_path_history, HOLE_BUFFER, FOOTPRINT_FRONT, FOOTPRINT_BACK, FOOTPRINT_SIDE, FOOTPRINT_BUFFER
    
    # Update Globals from Sliders (Real-time update)
    HOLE_BUFFER = hole_buffer_slider.value
    FOOTPRINT_FRONT = fp_front_slider.value
    FOOTPRINT_BACK = fp_back_slider.value
    FOOTPRINT_SIDE = fp_side_slider.value
    FOOTPRINT_BUFFER = fp_buffer_slider.value
    
    with out:
        clear_output(wait=True)
        # Get current hole positions
        holes = get_holes(h1_slider.value, h2_slider.value, h3_slider.value)
        
        # Create subplots: 3 rows, 1 column (Route, Evolution, Loss)
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 18), gridspec_kw={'height_ratios': [2, 2, 1]})
        
        # --- Plot 1: Interactive Route ---
        streets.plot(ax=ax1, color='blue', linestyle='--', label='Original Street')
        
        if current_fitted_streets is not None:
             current_fitted_streets.plot(ax=ax1, color='orange', linewidth=2, alpha=0.5, label='Last Fitted Street')
             plot_footprints(current_fitted_streets.geometry[0], ax1, step=5)
             
             # Dynamic Footprint
             line = current_fitted_streets.geometry[0]
             dist = line.length * path_slider.value
             point = line.interpolate(dist)
             delta = 0.1
             p_next = line.interpolate(min(dist + delta, line.length))
             p_prev = line.interpolate(max(dist - delta, 0))
             angle = math.atan2(p_next.y - p_prev.y, p_next.x - p_prev.x)
             
             real_fp = gen_footprint_obstacle(point.x, point.y, angle, buffer_val=0.0)
             ax1.plot(*real_fp.exterior.xy, color='black', linewidth=2, label='Test Robot')
             
             if FOOTPRINT_BUFFER > 0:
                 buf_fp = gen_footprint_obstacle(point.x, point.y, angle)
                 ax1.plot(*buf_fp.exterior.xy, color='black', linestyle='--', linewidth=1.5, label='Test Buffer')
        
        holes.plot(ax=ax1, color='red', markersize=100, label='Holes')
        holes.buffer(HOLE_BUFFER).plot(ax=ax1, color='red', alpha=0.2, label='Buffer')
        geofence.boundary.plot(ax=ax1, color='green', label='Geofence')
        ax1.set_title("Interactive Route Inspection")
        ax1.set_ylim(-20, 20)
        ax1.set_xlim(-10, 110)
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)
        
        # --- Plot 2: Evolution by Epochs ---
        streets.plot(ax=ax2, color='blue', linestyle='--', alpha=0.3)
        holes.plot(ax=ax2, color='red', markersize=100)
        holes.buffer(HOLE_BUFFER).plot(ax=ax2, color='red', alpha=0.1)
        geofence.boundary.plot(ax=ax2, color='green', alpha=0.3)
        
        if current_path_history:
            num_paths = len(current_path_history)
            for i, (px, py) in enumerate(current_path_history):
                # Gradient color: Light Orange -> Dark Orange
                alpha = 0.2 + 0.8 * (i / num_paths)
                color = (1.0, 0.5 * (1 - i/num_paths), 0.0) # Orange gradient
                ax2.plot(px, py, color=color, alpha=alpha, linewidth=1.5, label=f'Epoch {i*50}' if i == 0 or i == num_paths-1 else "")
            
            ax2.set_title(f"Path Evolution (Epochs 0 to {len(current_path_history)*50})")
        else:
            ax2.text(0.5, 0.5, "No history available", ha='center', va='center')
            ax2.set_title("Path Evolution")
            
        ax2.set_ylim(-20, 20)
        ax2.set_xlim(-10, 110)
        ax2.grid(True, alpha=0.3)

        # --- Plot 3: Loss ---
        if current_losses:
            ax3.plot(current_losses, color='blue', linewidth=1.5)
            ax3.set_title("Optimization Loss")
            ax3.set_xlabel("Iteration")
            ax3.set_ylabel("Loss")
            ax3.grid(True, alpha=0.3)
            ax3.set_yscale('log')
        else:
            ax3.text(0.5, 0.5, "No optimization run yet", ha='center', va='center')
        
        plt.tight_layout()
        plt.show()

def run_optimization(b):
    try:
        # Immediate debug print
        with debug_out:
            print("Button clicked! Starting optimization...")
    
        global current_fitted_streets, current_losses, current_path_history, HOLE_BUFFER, FOOTPRINT_FRONT, FOOTPRINT_BACK, FOOTPRINT_SIDE, FOOTPRINT_BUFFER
        global REPULSION_WEIGHT, FIDELITY_WEIGHT, SAFETY_MARGIN, SOFT_LIMIT_RANGE, ITERATIONS
        
        # Update Globals from Sliders
        HOLE_BUFFER = hole_buffer_slider.value
        FOOTPRINT_FRONT = fp_front_slider.value
        FOOTPRINT_BACK = fp_back_slider.value
        FOOTPRINT_SIDE = fp_side_slider.value
        FOOTPRINT_BUFFER = fp_buffer_slider.value
        
        # Update Tuning Globals
        REPULSION_WEIGHT = repulsion_slider.value
        FIDELITY_WEIGHT = fidelity_slider.value
        SAFETY_MARGIN = safety_margin_slider.value
        SOFT_LIMIT_RANGE = soft_limit_slider.value
        ITERATIONS = iterations_slider.value
        
        with out:
            clear_output(wait=True)
            print("Optimizing... please wait.")
            
            holes = get_holes(h1_slider.value, h2_slider.value, h3_slider.value)
            
            # fit_all_streets now returns (streets, losses, path_history)
            fitted_streets, losses, path_hist = fit_all_streets(streets.copy(), holes, geofence, None, None, fit_twice=True)
            current_fitted_streets = fitted_streets 
            current_losses = losses
            current_path_history = path_hist
            
            clear_output(wait=True) # Clear the "Optimizing..." message
            
            # Re-use plot_current_state to show results
            plot_current_state()
            
            with debug_out:
                print("Optimization finished successfully.")
            
    except Exception as e:
        with debug_out:
            print("Optimization failed!")
            print(traceback.format_exc())

# Link sliders to preview
h1_slider.observe(plot_current_state, names='value')
h2_slider.observe(plot_current_state, names='value')
h3_slider.observe(plot_current_state, names='value')
hole_buffer_slider.observe(plot_current_state, names='value')
fp_front_slider.observe(plot_current_state, names='value')
fp_back_slider.observe(plot_current_state, names='value')
fp_side_slider.observe(plot_current_state, names='value')
fp_buffer_slider.observe(plot_current_state, names='value')
path_slider.observe(plot_current_state, names='value')

# Layout
hole_controls = widgets.VBox([h1_slider, h2_slider, h3_slider])
param_controls = widgets.VBox([hole_buffer_slider, fp_front_slider, fp_back_slider, fp_side_slider, fp_buffer_slider])
tuning_controls = widgets.VBox([repulsion_slider, fidelity_slider, safety_margin_slider, soft_limit_slider, iterations_slider])
path_control = widgets.VBox([path_slider])

ui = widgets.HBox([hole_controls, param_controls, tuning_controls, path_control])

# Initial display
run_btn.on_click(run_optimization)
display(widgets.VBox([ui, run_btn, out, debug_out]))

# Run initial optimization so the user sees something immediately
run_optimization(None)
